In [18]:
cd /home/gxu/AllAMReX/FHDeX/exec/immersed_boundary/flagellum/

/home/gxu/AllAMReX/FHDeX/exec/immersed_boundary/flagellum


In [19]:
import sys, os, os.path
import glob

import scipy as sp
import numpy as np

import matplotlib
import matplotlib.pyplot as pp

import yt
from yt.frontends.boxlib.data_structures import AMReXDataset

In [20]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [21]:
class SoA:
    _pref = "particle_"
    _pos  = "position_"
    _vel  = "vel"
    # New here
    _id   = "id"
    _cpu  = "cpu"
    _id_0 = "id_0"
    _cpu_0 = "cpu_0"
    
    def __init__(self, data):
        str_pos = self._pref+self._pos
        self.px = np.array(data[str_pos + "x"])
        self.py = np.array(data[str_pos + "y"])
        self.pz = np.array(data[str_pos + "z"])

        str_vel = self._pref+self._vel
        self.vx = np.array(data[str_vel + "x"])
        self.vy = np.array(data[str_vel + "y"])
        self.vz = np.array(data[str_vel + "z"])

        str_id = self._pref+self._id
        self.id = np.array(data[str_id])
        str_cpu = self._pref+self._cpu
        self.cpu = np.array(data[str_cpu])
        
        str_id_0 = self._pref+self._id_0
        self.id_0 = np.array(data[str_id_0])
        str_cpu_0 = self._pref+self._cpu_0
        self.cpu_0 = np.array(data[str_cpu_0])
 
    # new here
    def __str__(self):
        return "{pos:"  + str(self.px) + "," + str(self.py) + "," + str(self.pz) + \
              "; vel:"  + str(self.vx) + "," + str(self.vy) + "," + str(self.vz) + \
              "; id:"   + str(self.id) + ", cpu:" + str(self.cpu) + ", id_0:" + str(self.id_0) + ", cpu_0:" + str(self.cpu_0) + "," + "}"


    def __repr__(self):
        return str(self)


# new here
class Particle:
    def __init__(self, px, py, pz, vx, vy, vz, id, cpu, id_0, cpu_0):
        self.pos = np.array([px, py, pz])
        self.vel = np.array([vx, vy, vz])
        self.id  = np.array(id)
        self.cpu  = np.array(cpu)
        self.id_0  = np.array(id_0)
        self.cpu_0  = np.array(cpu_0)

    def __str__(self):
        return "P(" + str(self.pos) + "," + str(self.vel) + "," + str(self.id) + "," + str(self.cpu) + "," + str(self.id_0) + "," + str(self.cpu_0) + "," + ")"
    

    def __repr__(self):
        return str(self)


class AoS:
    def __init__(self, amrex_data):
        self.particles = list()
        soa = SoA(amrex_data)
        # new here        
        data = zip(soa.px, soa.py, soa.pz, soa.vx, soa.vy, soa.vz, soa.id, soa.cpu, soa.id_0, soa.cpu_0)
        for elt in data:
            self.particles.append(Particle(* elt))




In [22]:
data_root = "."
data_dir  = "."

In [23]:
data_path = os.path.join(data_root, data_dir)

In [24]:
n_fill   = 5
prefix   = "plt"
file_fmt = prefix + "{:0" + str(n_fill) + "d}"

In [30]:
data_glob  = os.path.join(data_path, prefix + "*")
data_files = glob.glob(data_glob)
data_files.sort()
print(data_files)

['././plt0000000', '././plt0000010', '././plt0000020', '././plt0000030', '././plt0000040', '././plt0000050', '././plt0000060', '././plt0000070', '././plt0000080', '././plt0000090', '././plt0000100', '././plt0000110', '././plt0000120', '././plt0000130', '././plt0000140', '././plt0000150', '././plt0000160', '././plt0000170', '././plt0000180', '././plt0000190', '././plt0000200', '././plt0000210', '././plt0000220', '././plt0000230', '././plt0000240', '././plt0000250', '././plt0000260', '././plt0000270', '././plt0000280', '././plt0000290', '././plt0000300', '././plt0000310', '././plt0000320', '././plt0000330', '././plt0000340', '././plt0000350', '././plt0000360', '././plt0000370', '././plt0000380', '././plt0000390', '././plt0000400', '././plt0000410', '././plt0000420', '././plt0000430', '././plt0000440', '././plt0000450', '././plt0000460', '././plt0000470', '././plt0000480', '././plt0000490', '././plt0000500', '././plt0000510', '././plt0000520', '././plt0000530', '././plt0000540', '././plt0

In [31]:
len(data_files)

101

In [34]:
#positions_ = dict()


for i, data_file in enumerate(data_files):
    ds = yt.load(data_file)
    
    amrex_ds = AMReXDataset(data_file)
    ad = amrex_ds.all_data()
    aos = AoS(ad)
    
        
    slc = yt.SlicePlot(ds, "z", "averaged_velz")
    
      
    slc.set_log("averaged_velx", False)
    slc.set_log("averaged_vely", False)
    slc.set_log("averaged_velz", False)
    slc.set_zlim('averaged_velz', -.025, .025)
    
    slc.annotate_quiver("averaged_velx", "averaged_vely")
    
    for j, part in enumerate(aos.particles):
        p = part.pos
        p.tolist()
        slc.annotate_sphere(p, radius=(0.01), circle_args={'color':'white'})
        #slc.annotate_marker(p)

        slc.save("frame_" + str(i).zfill(5) + ".png")

yt : [INFO     ] 2019-07-17 13:32:09,811 Parameters: current_time              = 0.0
yt : [INFO     ] 2019-07-17 13:32:09,816 Parameters: domain_dimensions         = [32 32 32]
yt : [INFO     ] 2019-07-17 13:32:09,818 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2019-07-17 13:32:09,820 Parameters: domain_right_edge         = [1. 1. 1.]
yt : [INFO     ] 2019-07-17 13:32:09,859 Parameters: current_time              = 0.0
yt : [INFO     ] 2019-07-17 13:32:09,861 Parameters: domain_dimensions         = [32 32 32]
yt : [INFO     ] 2019-07-17 13:32:09,862 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2019-07-17 13:32:09,864 Parameters: domain_right_edge         = [1. 1. 1.]
yt : [INFO     ] 2019-07-17 13:32:10,400 xlim = 0.000000 1.000000
yt : [INFO     ] 2019-07-17 13:32:10,402 ylim = 0.000000 1.000000
yt : [INFO     ] 2019-07-17 13:32:10,405 xlim = 0.000000 1.000000
yt : [INFO     ] 2019-07-17 13:32:10,407 ylim = 0.000000 1.000000
yt : [INFO    

In [33]:
ds.field_list

[('boxlib', 'averaged_velx'),
 ('boxlib', 'averaged_vely'),
 ('boxlib', 'averaged_velz'),
 ('boxlib', 'divergence'),
 ('boxlib', 'pres'),
 ('boxlib', 'shifted_velx'),
 ('boxlib', 'shifted_vely'),
 ('boxlib', 'shifted_velz'),
 ('boxlib', 'tracer')]

In [ ]:
ds.particle_field_by_type


In [ ]:
ad = ds.all_data()
aos = AoS(ad)

In [ ]:
for i, part in enumerate(aos.particles):
    print(part.pos)

In [ ]:
p = aos.particles[1]


In [ ]:
p.annotate_sphere([0.5, 0.5, 0.5], radius=(2, 'kpc'),
                  circle_args={'color':'black'})

In [ ]:
p


In [ ]:
#ffmpeg -pattern_type glob -i "fame_names_*.png" -r 24 -c:v libx264 video.mpeg